In [ ]:
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
from keras.models import Model
import os
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import train_test_split




In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

#bands = [6, 7, 20, 28, 28, 31]


1 Physical GPUs, 1 Logical GPUs


2023-09-22 18:53:11.820350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21020 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [ ]:
### PROCESSING .hdf files
import extract_training_data
import importlib
importlib.reload(extract_training_data)
from extract_training_data import *
#folder = "/uio/hume/student-u37/fslippe/data/training_data/MOD02QKM/normalized_data/"
folder = "/scratch/fslippe/modis/MOD02/daytime_1km/"

#folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02QKM_202012-202104/normalized_data/"

start = "20201201"
end = "20210430"
#dates = ["20210303"]#, "20210322"]#, "20210323"]
#dates_converted = []
#for date in dates:
#    dates_converted.append(convert_to_day_of_year(date))

start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
#print(start_converted)
#print(end_converted)
#x = [xi for xi in  extract_250m_data(folder, bands=[1], date_list=dates_converted) if xi.shape[0] > 256]
x = [xi for xi in  extract_1km_data(folder, bands=[1], start_date=start_converted, end_date=end_converted) if xi.shape[0] > 64]
hdf = SD("/scratch/fslippe/modis/MOD02/daytime_1km/MOD021KM.A2021096.1620.061.2021097013938.hdf", SDC.READ)

len(x)
#x = extract_250m_data(folder, bands=[1], start_date=start_converted, end_date=end_converted)


/scratch/fslippe/modis/MOD02/daytime_1km/MOD021KM.A2020336.0800.061.2020337214455.hdf
0 Latitude
1 Longitude
2 EV_1KM_RefSB
3 EV_1KM_RefSB_Uncert_Indexes
4 EV_1KM_Emissive
5 EV_1KM_Emissive_Uncert_Indexes
6 EV_250_Aggr1km_RefSB
7 EV_250_Aggr1km_RefSB_Uncert_Indexes
8 EV_250_Aggr1km_RefSB_Samples_Used
9 EV_500_Aggr1km_RefSB
10 EV_500_Aggr1km_RefSB_Uncert_Indexes
11 EV_500_Aggr1km_RefSB_Samples_Used
12 Height
13 SensorZenith
14 SensorAzimuth
15 Range
16 SolarZenith
17 SolarAzimuth
18 gflags
19 EV_Band26
20 EV_Band26_Uncert_Indexes
21 Band_250M
22 Band_500M
23 Band_1KM_RefSB
24 Band_1KM_Emissive
25 Noise in Thermal Detectors
26 Change in relative responses of thermal detectors
27 DC Restore Change for Thermal Bands
28 DC Restore Change for Reflective 250m Bands
29 DC Restore Change for Reflective 500m Bands
30 DC Restore Change for Reflective 1km Bands


/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:10<00:00, 11.63it/s]


732

In [ ]:
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
bands = [1]
patch_size = 64

autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
#x = autoencoder.normalize(x)
#optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=1e-4), loss_scale='dynamic')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model = autoencoder.model(optimizer=optimizer, threshold=0.09,loss="combined")


Input should already be normalized. Call self.normalize to normalize list of data


In [ ]:
patches = np.concatenate([autoencoder.extract_patches(n_d) for n_d in x], axis=0)


In [ ]:
# Splitting the data
patches, val_data = train_test_split(patches, test_size=0.20, random_state=42)
print("finished split")

finished split


In [ ]:
print(patches.shape)

(378268, 64, 64, 1)


In [ ]:
min_val = 0
max_val = np.max(patches)
print(max_val)
patches = (patches - min_val) / (max_val - min_val)
val_data = (val_data - min_val) / (max_val - min_val)

539.9074968807399


In [ ]:
gc.collect()

194

In [ ]:
model.fit(patches, patches, validation_data=(val_data, val_data), epochs=100, batch_size=32)


2023-09-22 16:36:06.316191: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


Epoch 1/100


2023-09-22 16:36:11.217255: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


11821/11821 [==============================] - 71s 6ms/step - loss: 0.0067 - val_loss: 0.0046
Epoch 2/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 3/100
11821/11821 [==============================] - 63s 5ms/step - loss: 0.0033 - val_loss: 0.0029
Epoch 4/100
11821/11821 [==============================] - 63s 5ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 5/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 6/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 7/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0023 - val_loss: 0.0022
Epoch 8/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 9/100
11821/11821 [==============================] - 64s 5ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 10/100
11821/11821 [==============================] 

In [ ]:
#model.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_autoencoder")
autoencoder.encoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_encoder")
autoencoder.decoder.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_decoder")


INFO:tensorflow:Assets written to: /uio/hume/student-u37/fslippe/data/models/winter_2020_21_encoder/assets


/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


INFO:tensorflow:Assets written to: /uio/hume/student-u37/fslippe/data/models/winter_2020_21_decoder/assets


In [ ]:
np.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_val_patches", val_data)
np.save("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_train_patches", patches)


In [ ]:
from autoencoder import SobelFilterLayer, SimpleAutoencoder
bands = [1]
patch_size = 256
from tensorflow.keras.models import load_model
print(len(bands))
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

encoder = load_model("/uio/hume/student-u37/fslippe/data/models/test_1day_encoder")

In [ ]:
# normalized_patches = np.concatenate([autoencoder.extract_patches(n_d) for n_d in normalized_data[:4]], axis=0)

start = 3
index_list = [3, 10, 11, 12, 13]
cluster_map = autoencoder_predict.kmeans([normalized_data[i] for i in index_list], n_clusters=10, encoder=encoder)


In [ ]:
index_list = [3, 7, 8, 9]
print(np.max(normalized_data[8][:,:,0]))
print(np.max(normalized_data[3][:,:,0]))
print(np.where(normalized_data[3][:,:,0] <0.8))


In [ ]:
from matplotlib.colors import Normalize

# Determine global min and max labels
global_min = np.min([np.min(cm) for cm in cluster_map])
global_max = np.max([np.max(cm) for cm in cluster_map])

norm = Normalize(vmin=global_min, vmax=global_max)

for i in range(len(cluster_map)):
    fig, axs = plt.subplots(1, 2, figsize=[10, 8])
    
    cb = axs[0].imshow(cluster_map[i], cmap="tab10", norm=norm)
    plt.colorbar(cb, ax=axs[0])
    
    cb = axs[1].imshow(normalized_data[index_list[i]][::4, ::4, 0])
    plt.colorbar(cb, ax=axs[1])

    plt.tight_layout()


In [ ]:
folder = "/uio/hume/student-u37/fslippe/data/cao/"
i=0
print(folder + all_files[i])
hdf = SD(folder + all_files[i], SDC.READ)
print(hdf.attributes())
lon = hdf.select('Longitude')[:]
lat = hdf.select('Latitude')[:]
downsample_factor_x = int(lon.shape[0] / cluster_map[i].shape[0])
downsample_factor_y = int(lon.shape[1] / cluster_map[i].shape[1])
cluster_shape = cluster_map[i].shape
lon_d = lon[::downsample_factor_x, ::downsample_factor_y][:cluster_shape[0], :cluster_shape[1]]
lat_d = lat[::downsample_factor_x, ::downsample_factor_y][:cluster_shape[0], :cluster_shape[1]]
data = hdf.select("EV_250_Aggr1km_RefSB")[:][0]
print(data.shape)
hdf.select("EV_250_Aggr1km_RefSB").attributes()
print(lon.shape)
cluster_map[i].shape
datasets = hdf.datasets()   
for idx, sds in enumerate(datasets.keys()):
    print(idx, sds)




In [ ]:
fix, axs= plt.subplots(1,2)
axs[0].imshow(cluster_map[i])
axs[1].imshow(X[i][:,:,0])


In [ ]:
projection = ccrs.NorthPolarStereo()
fig, axs = plt.subplots(1, 2, figsize=[10,5], subplot_kw={'projection': projection}, dpi=300)
cb = axs[0].pcolormesh(lon_d, lat_d, cluster_map[i], transform=ccrs.PlateCarree(), cmap="jet")  
axs[0].coastlines()
gl = axs[0].gridlines(draw_labels=True)
plt.colorbar(cb)

cb = axs[1].pcolormesh(lon, lat, data, transform=ccrs.PlateCarree(), vmin=0)  
axs[1].coastlines()
gl = axs[1].gridlines(draw_labels=True)
plt.colorbar(cb)
plt.show()

In [ ]:
# patches_predict = autoencoder.extract_patches(dataset_5.reshape((1,dataset_5.shape[0], dataset_5.shape[1], 1)))
# predict = autoencoder.autoencoder.predict(patches_predict)
# predict.shape

# #for i in range(83, 90, 2):
# pictures = 12
# patch_start = 68
# fig, axs = plt.subplots(2, pictures, figsize=[pictures*2,5])
# i=0
# for patch_number in range(patch_start, patch_start+pictures):
#     axs[0, i].imshow(patches_predict[patch_number], cmap="gray")
#     axs[1, i].imshow(predict[patch_number,:,:,0], cmap="gray")
    
#     # Turn off ticks and axis labels for both x and y
#     axs[0, i].set_yticks([])
#     axs[0, i].set_xticks([])
#     axs[0, i].axis('off')
    
#     axs[1, i].set_yticks([])
#     axs[1, i].set_xticks([])
#     axs[1, i].axis('off')
    
#     i += 1
# plt.tight_layout()

patches_predict = autoencoder2.extract_patches(dataset_5.reshape((1,dataset_5.shape[0], dataset_5.shape[1], 1)))
predict = autoencoder2.autoencoder.predict(patches_predict)
print(predict.shape)

#for i in range(83, 90, 2):
pictures = 10
patch_start = 320
fig, axs = plt.subplots(2, pictures, figsize=[pictures*2,5])
i=0
for patch_number in range(patch_start, patch_start+pictures):
    axs[0, i].imshow(patches_predict[patch_number], cmap="gray")
    axs[1, i].imshow(predict[patch_number,:,:,0], cmap="gray")
    
    # Turn off ticks and axis labels for both x and y
    axs[0, i].set_yticks([])
    axs[0, i].set_xticks([])
    axs[0, i].axis('off')
    
    axs[1, i].set_yticks([])
    axs[1, i].set_xticks([])
    axs[1, i].axis('off')
    
    i += 1
plt.tight_layout()

plt.figure()
plt.imshow(dataset_5, cmap='gray')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

cmp = axes[0].imshow(dataset_2, cmap='gray')
plt.colorbar(cmp, ax=axes[0])
axes[0].set_title('Original Image')
axes[0].axis('off')

cmp = axes[1].imshow(labels_img, cmap='tab10')
plt.colorbar(cmp, ax=axes[1])
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()

In [ ]:
encoder = Model(flattened_input, encoded)
encoded_imgs = encoder.predict(data)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Reduce the encoded images to 2D using PCA
pca = PCA(n_components=2)
encoded_imgs_2d = pca.fit_transform(encoded_imgs)

# Now cluster the 2D encoded images
n_clusters = 5  # Define the number of clusters you want
kmeans = KMeans(n_clusters=n_clusters)
cluster_assignments = kmeans.fit_predict(encoded_imgs_2d)

print(cluster_assignments.shape)

# If you want to visualize the clusters using matplotlib:
plt.scatter(encoded_imgs_2d[:, 0], encoded_imgs_2d[:, 1], c=cluster_assignments, cmap='jet')
plt.colorbar()
plt.title('Clusters in 2D')
plt.show()


In [ ]:
decoded_imgs = autoencoder.predict(dataset_2)
fig, axs = plt.subplots(1, 2, figsize=[10,5])

cmp = axs[0].imshow(dataset_2, cmap="gray")
plt.colorbar(cmp)
cmp = axs[1].imshow(decoded_imgs,cmap="gray")
plt.colorbar(cmp)

#decoded_imgs_2d = decoded_imgs.reshape((-1, *input_shape))
# plt.contourf(decoded_imgs_2d)
# plt.show(


In [ ]:
decoded_imgs_reshaped = decoded_imgs.reshape(-1, 1)

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(decoded_imgs_reshaped)
labels = kmeans.labels_
labels_reshaped = labels.reshape(2040, 1354)

# Assuming `original_img` is your original image
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

cmp = axes[0].imshow(dataset_2, cmap='gray')  # or just cmap depending on the nature of your image
plt.colorbar(cmp)
axes[0].set_title('Original Image')
axes[0].axis('off')

cmp = axes[1].imshow(labels_reshaped, cmap='tab10')  # 'tab10' is a colormap with distinct colors
plt.colorbar(cmp)
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()


In [ ]:

# Parameters
block_size = (10, 10)  # Change accordingly
n_clusters = 5

# Divide the image into blocks and calculate the mean for each block
blocks = []
for i in range(0, decoded_imgs.shape[0], block_size[0]):
    for j in range(0, decoded_imgs.shape[1], block_size[1]):
        block = decoded_imgs[i:i+block_size[0], j:j+block_size[1]]
        block_mean = np.mean(block)
        blocks.append(block_mean)

blocks = np.array(blocks).reshape(-1, 1)

# Cluster the blocks
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(blocks)
labels = kmeans.labels_

# Construct the clustered image
clustered_img = np.zeros_like(decoded_imgs)
label_idx = 0
for i in range(0, decoded_imgs.shape[0], block_size[0]):
    for j in range(0, decoded_imgs.shape[1], block_size[1]):
        clustered_img[i:i+block_size[0], j:j+block_size[1]] = labels[label_idx]
        label_idx += 1

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Assuming `original_img` is your original image
axes[0].imshow(dataset_2, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(clustered_img, cmap='tab10')  # 'tab10' for distinct colors
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()